# Non-central student $t$ fit
In this notebook we fit a location-scale non-central student $t$ to the stock returns

In [1]:
import numpy as np 
import pandas as pd
from scipy.stats import nct

stocks = np.load('data/plr_stocks.npy')

In [2]:
stock = stocks[0]
print(stock)

params = nct.fit(stock)
print(params)

[ 0.66667161  0.34546973  0.31788133 ...  0.64412684 -0.93994314
  0.8061777 ]
(np.float64(3.244760942387329), np.float64(-0.0596978223975496), np.float64(0.19421049769756688), np.float64(1.7630659317076822))


In [3]:
results = []
i = 1
k = 4 # all non-central student t fits have k=4 parameters
# (location, scale, df and non-centrality param)

# loop iterating over all stocks
for stock in stocks:

    df_fit, nc_fit, mu_fit, sigma_fit = nct.fit(stock)

    log_likelihood = nct.logpdf(stock, df = df_fit, nc = nc_fit, 
                                loc = mu_fit, scale = sigma_fit).sum()
    
    aic = 2*k - 2*log_likelihood
    bic = (k * np.log(len(stock))) - 2*log_likelihood
    
    results.append({'stock': i, 'df':df_fit, 'nc': nc_fit,
                    'mu': mu_fit, 'sigma':sigma_fit,
                    'log-likelihood' : log_likelihood,
                    'AIC':aic, 'BIC': bic})
    i += 1 
    
results = pd.DataFrame(results)
print(results.head())
results.to_csv('results/nct_fit.csv', index=False)

   stock        df        nc        mu     sigma  log-likelihood  \
0      1  3.244761 -0.059698  0.194210  1.763066   -18503.260154   
1      2  2.688342 -0.075064  0.159506  1.295517   -16605.770416   
2      3  3.261240  0.028082 -0.006172  1.320783   -16177.164857   
3      4  3.778361 -0.039010  0.109096  1.428540   -16436.155634   
4      5  3.944277 -0.107353  0.179055  1.129174   -14461.196367   

            AIC           BIC  
0  37014.520307  37042.469095  
1  33219.540832  33247.489619  
2  32362.329714  32390.278502  
3  32880.311267  32908.260054  
4  28930.392733  28958.341520  


# Value at Risk estimation

In [4]:
#print(results.head())
#print(results.iloc[0]['mu'])

results_var = []

for i in range(results.shape[0]):

    df_hat = results.iloc[i]['df']
    nc_hat = results.iloc[i]['nc']
    mu_hat = results.iloc[i]['mu']
    sigma_hat = results.iloc[i]['sigma']

    alpha = 0.01

    empirical_var = -np.percentile(stocks[i], alpha * 100)
    var = -nct.ppf(alpha, df = df_hat, nc = nc_hat, loc = mu_hat, scale = sigma_hat)

    results_var.append({'stock': i + 1,'estimated-var':var, 'empirical-var':empirical_var})

    
results_var = pd.DataFrame(results_var)
results_var.head()
results_var.to_csv('results/ValueAtRiskResults/nct_var.csv', index=False)
